In [67]:
import json
import argparse
import torch
import numpy as np
from tqdm import tqdm
import evaluate
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
    
device = "cuda:0" if torch.cuda.is_available() else "cpu"    

class Experiment:

    def __init__(self, learning_rate, model, epochs, batch_size, neg, dataset_path):
        self.model_name = model
        self.dataset_path = dataset_path
        self.learning_rate = learning_rate
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")#"model"
        self.dataset = self.process_dataset(dataset_path, neg)
        self.tokenized_datasets = self.dataset.map(self.tokenize_function, batched=True)
        self.model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=2).to(device)
        self.metric = evaluate.load("glue", "mrpc")
        self.training_args = TrainingArguments(
                output_dir="output/"+self.model_name,
                logging_steps = 500,
                evaluation_strategy="steps",
                eval_steps = 500,
                num_train_epochs = epochs,
                learning_rate = learning_rate,
                per_device_train_batch_size = batch_size
                )

    def process_dataset(self, dataset_path, neg):
        #convert dataset into json for dataset loader
        d_file = open(dataset_path, 'r')
        d_json = json.load(d_file)
        formatted_examples = []
        for example in tqdm(d_json, desc="Loading Dataset"):
            #create an entry for each positive example
            positive_ids = list(np.array(example["positive_idxs"]) - 1)
            candidate = [example["derivation"][equation_id][1] for equation_id in positive_ids]
            context = (str(example['derivation'][:-1]) + ' [SEP] ' + str(example['derivation'][-1][0])).replace('[[','[').replace(']]',']').replace('\\\\','\\')
            input_text = context + " [SEP] " + " ".join(candidate)
            formatted_examples.append({"text": input_text, "label": 1})
            #create an entry for each negative example
            count_neg = 0
            for negative in example["negatives"]:
                if count_neg == neg:
                    break
                input_text = context + " [SEP] " + negative
                formatted_examples.append({"text": input_text, 'label': 0})
                count_neg += 1
        print("Data examples", formatted_examples[:4])
        #split randomly between train, dev, and test set
        dataset = Dataset.from_list(formatted_examples)
        dataset_split = dataset.train_test_split(test_size=0.99)
        return dataset_split

    def tokenize_function(self, examples):
        return self.tokenizer(examples["text"], padding="max_length", truncation=True)

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        majority_class_preds = [1 for pred in predictions]
        majority_baseline_score = self.metric.compute(predictions=majority_class_preds, references=labels)
        print("majority_class_baseline:", majority_baseline_score)
        score = self.metric.compute(predictions=predictions, references=labels)
        print(score)
        return score

    def train_and_eval(self):
        trainer = Trainer(
            model = self.model,
            args = self.training_args,
            train_dataset = self.tokenized_datasets["train"],
            eval_dataset = self.tokenized_datasets["test"],
            compute_metrics = self.compute_metrics
        )
        trainer.evaluate()

if __name__ == '__main__':
    
    dataset = "EVAL_next_equation_selection_OP_SWAP_steps=3.json"
    
    model = "models/bert-base-uncased/checkpoint-7000/"

    data_path = dataset
    torch.backends.cudnn.deterministic = True 
    seed = 42
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available:
        torch.cuda.manual_seed_all(seed)
    experiment = Experiment(
            learning_rate = 5e-5,
            batch_size = 8, 
            neg = 1, 
            epochs = 1, 
            model = model, 
            dataset_path = data_path
            )
    experiment.train_and_eval()
                


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\JCMea/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/r

Data examples [{'text': "['premise', 'h{(v)} = \\log{(v)}'], ['differentiate', 1, 'v'], '\\frac{d}{d v} h{(v)} = \\frac{d}{d v} \\log{(v)}'] [SEP] ['integrate', 2, 'v'] [SEP] \\int \\frac{d}{d v} h{(v)}\\, dv = \\int \\frac{d}{d v} \\log{(v)}\\, dv", 'label': 1}, {'text': "['premise', 'h{(v)} = \\log{(v)}'], ['differentiate', 1, 'v'], '\\frac{d}{d v} h{(v)} = \\frac{d}{d v} \\log{(v)}'] [SEP] ['integrate', 2, 'v'] [SEP] \\frac{d}{d v} h{(v)} = \\frac{1}{v}", 'label': 0}, {'text': "['premise', 'T{(l)} = \\cos{(l)}'], ['divide', 1, '\\cos{(l)}'], '\\frac{T{(l)}}{\\cos{(l)}} = 1'] [SEP] ['integrate', 2, 'l'] [SEP] \\int \\frac{T{(l)}}{\\cos{(l)}}\\, dl = \\int 1\\, dl", 'label': 1}, {'text': "['premise', 'T{(l)} = \\cos{(l)}'], ['divide', 1, '\\cos{(l)}'], '\\frac{T{(l)}}{\\cos{(l)}} = 1'] [SEP] ['integrate', 2, 'l'] [SEP] L{(l)} = \\cos{(l)}", 'label': 0}]


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

loading configuration file models/bert-base-uncased/checkpoint-7000/config.json
Model config BertConfig {
  "_name_or_path": "models/bert-base-uncased/checkpoint-7000/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file models/bert-base-uncased/checkpoint-7000/pytorch_model.bin
All model checkpoint weights were used when initializing BertFor

majority_class_baseline: {'accuracy': 0.5010940919037199, 'f1': 0.6676384839650146}
{'accuracy': 0.5448577680525164, 'f1': 0.6040609137055838}
